In [13]:
import requests
import mwparserfromhell
import csv
from bs4 import BeautifulSoup

In [14]:
def fetch_pages(category_title):
    url = "https://bots.snpedia.com/api.php"
    params = {
        "action" : "query",
        "list" : "categorymembers",
        "cmtitle" : category_title,
        "cmlimit" : "max",
        "format" : "json",
    }

    response = requests.get(url, params=params)
    data = response.json()

    pages = data["query"]["categorymembers"]

    return pages

In [15]:
def fetch_page_content_by_title(page_title):
    url = "https://bots.snpedia.com/api.php"
    params = {
        "action": "query",
        "titles": page_title,
        "prop": "revisions",
        "rvprop": "content",
        "format": "json",
    }

    response = requests.get(url, params=params)
    data = response.json()

    # Get the first page ID from the response
    page_id = list(data["query"]["pages"].keys())[0]

    try:
        content = data["query"]["pages"][page_id]["revisions"][0]["*"]
    except KeyError:
        content = "Content not found"

    return content

In [16]:
def follow_links(wikitext):
    # Parse the wikitext using mwparserfromhell
    wikicode = mwparserfromhell.parse(wikitext)
    
    # Find all internal links
    links = wikicode.filter_wikilinks()

    for link in links:
        # Check if the link starts with "/index.php/"
        if str(link).startswith("[[/index.php/"):
            # Extract the page title from the link
            page_title = str(link)[12:-1]

            # Fetch the content of the linked page
            print(f"Fetching content for: {page_title}")
            content = fetch_page_content_by_title(page_title)
            print(f"Content: {content}\n")